## 참고 사이트
- https://velog.io/@ezoo0422/Python-pycaret%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EB%AA%A8%EB%8D%B8-%EC%84%A0%EC%A0%95%ED%95%98%EA%B8%B0
- https://ideal-dominate.medium.com/pycaret-regression-%EA%B0%99%EC%9D%B4-%ED%95%B4%EB%B4%90%EC%9A%94-eaf0e36268
- https://ideal-dominate.medium.com/%ED%8C%8C%EB%9E%80%EB%A7%8C%EC%9E%A5-pycaret-%EC%84%A4%EC%B9%98%EA%B8%B0-cf703f692e3a
- https://koreapy.tistory.com/744

- https://pycaret.gitbook.io/docs/get-started/quickstart

## pycaret 설치
- conda install -c conda-forge pycaret

In [1]:
#pip install pycaret[full]

## data load

In [2]:
# from pycaret.datasets import get_data
# dataset = get_data('insurance')

random state를 789으로 해줘서 90% :10%으로 훈련 데이터, 테스트 데이터를 나눈다

# 2012-2015년 데이터로 모델 만들기

In [3]:
import pandas as pd

In [4]:
dataset2 = pd.read_csv('./hospital_male_0725.csv', encoding='cp949')
dataset2.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
dataset2 = dataset2[['area','yyyy','mm','dd','frequency','elderly_pop','total_pop','elderly_ratio','avg_age','pm10','avg_rhm','rhm_min_avg_dif','avg_ta','min_max_Ta','avg_ps','min_max_ps']]
dataset2

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2012.0,1.0,1.0,3,96281.0,772718.0,12.460044,39.4,86.279167,71.285714,23.088571,-4.226027,9.280556,1023.886667,6.340000
1,경기,2012.0,1.0,1.0,4,447450.0,6020446.0,7.432174,36.1,91.567357,73.827586,25.065517,-3.962264,9.245283,1024.875000,5.500000
2,경남,2012.0,1.0,1.0,2,152866.0,1665308.0,9.179443,37.4,59.554398,63.333333,19.133333,0.335897,10.074359,1022.950000,5.200000
3,경북,2012.0,1.0,1.0,6,167115.0,1354649.0,12.336406,39.5,76.618056,65.052632,17.005263,-0.666667,9.628070,1023.268000,5.688000
4,광주,2012.0,1.0,1.0,0,55792.0,726529.0,7.679253,35.3,44.745833,86.333333,15.400000,-0.200000,6.500000,1025.250000,4.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,2015.0,12.0,31.0,0,154519.0,953881.0,16.198981,42.1,55.015673,80.300000,24.265854,4.156757,7.135135,1031.053846,6.892308
24833,전북,2015.0,12.0,31.0,1,135253.0,930255.0,14.539347,40.9,69.055014,86.266667,25.910417,1.583333,7.536667,1031.300000,7.014286
24834,제주,2015.0,12.0,31.0,0,34676.0,313428.0,11.063466,38.7,71.968750,73.903226,21.264516,5.128571,4.877143,1030.792308,7.523077
24835,충남,2015.0,12.0,31.0,0,142107.0,1054439.0,13.477024,40.2,51.015625,79.647059,24.624837,1.912121,7.154545,1031.513333,6.800000


In [5]:
data = dataset2.sample(frac=0.9, random_state=786) 
# data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
data_unseen = dataset2.drop(data.index) 
data.reset_index(drop=True, inplace=True) 
data_unseen.reset_index(drop=True, inplace=True) 
print('Data for Modeling: ' + str(data.shape)) 
print('Unseen Data For Predictions: ' + str(data_unseen.shape)) 
# Data for Modeling: (5400, 8) # Unseen Data For Predictions: (600, 8)

Data for Modeling: (22353, 16)
Unseen Data For Predictions: (2484, 16)


## 모델 환경 설정 (Settinf up Environment in PyCaret)

- pycaret을 사용하기 전에 pycaret에 맞게 데이터를 설정
- set_up() 함수를 사용
- 기본적으로 data와 target을 입력

In [6]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
s = setup(data, target = 'frequency')

,Description,Value
0,Session id,8699
1,Target,frequency
2,Target type,Regression
3,Data shape,"(22353, 16)"
4,Train data shape,"(15647, 16)"
5,Test data shape,"(6706, 16)"
6,Numeric features,14
7,Categorical features,1
8,Preprocess,True
9,Imputation type,simple


#### set_up(): pycaret을 사용하기 위한 data setting

- session_id: random_state와 같은 개념으로 같은 결과가 나올 수 있게 seed를 고정
- data: train 데이터 입력
- target = target 변수 이름을 입력

## 모델 생성 - 비교

- 여러 모델을 적합하여 성능을 비교하는 단계
- compare_models(): 다양한 모델 적합 후 성능 비교
    - fold: cross_validation의 fold를 지정 (default = 10)
    - sort: 정렬기준 지표 설정
    - n_select: 상위 n개의 모델 결과만 출력

In [7]:
best_model_5 = compare_models(sort='RMSE', n_select = 5)
best_model_5

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.9049,1.5497,1.2443,0.4591,0.4881,0.4382,0.1570
gbr,Gradient Boosting Regressor,0.8997,1.5566,1.2471,0.4566,0.4906,0.4497,0.4010
lr,Linear Regression,0.9075,1.5591,1.2480,0.4560,0.4882,0.4376,0.2160
ridge,Ridge Regression,0.9075,1.5591,1.2480,0.4560,0.4882,0.4376,0.0120
lar,Least Angle Regression,0.9074,1.5591,1.2480,0.4560,0.4882,0.4376,0.0120
br,Bayesian Ridge,0.9086,1.5593,1.2481,0.4559,0.4882,0.4378,0.0120
omp,Orthogonal Matching Pursuit,0.9053,1.5629,1.2496,0.4546,0.4881,0.4355,0.0110
lightgbm,Light Gradient Boosting Machine,0.8715,1.6058,1.2665,0.4395,0.4985,0.4258,0.0420
en,Elastic Net,0.9390,1.6070,1.2671,0.4393,0.4946,0.4295,0.0150
lasso,Lasso Regression,0.9394,1.6084,1.2676,0.4388,0.4947,0.4292,0.0190


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

[ExtraTreesRegressor(n_jobs=-1, random_state=8699),
 GradientBoostingRegressor(random_state=8699),
 LinearRegression(n_jobs=-1),
 Ridge(random_state=8699),
 Lars(random_state=8699)]

## 모델 튜닝 - blending
- 생성한 모델끼리의 혼합

In [8]:
blender_5 = blend_models(best_model_5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8936,1.5127,1.2299,0.4703,0.4799,0.4301
1,0.9195,1.5589,1.2486,0.4254,0.5003,0.4380
2,0.9116,1.5247,1.2348,0.4329,0.4880,0.4596
3,0.9058,1.5484,1.2444,0.4470,0.4850,0.4396
4,0.9204,1.6192,1.2725,0.4609,0.4894,0.4483
5,0.8471,1.3490,1.1615,0.4886,0.4850,0.4108
6,0.9380,1.7303,1.3154,0.4227,0.5049,0.4606
7,0.8953,1.4712,1.2129,0.4588,0.4857,0.4341
8,0.8890,1.5986,1.2644,0.4949,0.4767,0.4220


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

## 모델 튜닝 - ensemble
- 생성한 모델끼리의 혼합

In [14]:
bagging_5 = ensemble_model(best_model_5[0], method='Bagging')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8970,1.5181,1.2321,0.4684,0.4804,0.4293
1,0.9201,1.5534,1.2464,0.4274,0.5005,0.4361
2,0.9139,1.5186,1.2323,0.4352,0.4879,0.4629
3,0.9069,1.5415,1.2416,0.4495,0.4856,0.4383
4,0.9194,1.6037,1.2664,0.4661,0.4881,0.4502
5,0.8497,1.3523,1.1629,0.4874,0.4865,0.4123
6,0.9406,1.7230,1.3126,0.4252,0.5051,0.4610
7,0.8961,1.4663,1.2109,0.4606,0.4866,0.4349
8,0.8913,1.5943,1.2626,0.4963,0.4774,0.4217


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
boosting_5 = ensemble_model(best_model_5[0], method='Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.8973,1.5152,1.2309,0.4694,0.4804,0.4294
1,0.9229,1.5499,1.2449,0.4287,0.5009,0.4368
2,0.9150,1.5155,1.2311,0.4363,0.4887,0.4627
3,0.9081,1.5415,1.2416,0.4495,0.4866,0.4382
4,0.9212,1.5992,1.2646,0.4676,0.4891,0.4504
5,0.8490,1.3482,1.1611,0.4889,0.4861,0.4112
6,0.9412,1.7210,1.3119,0.4258,0.5052,0.4612
7,0.8985,1.4682,1.2117,0.4599,0.4881,0.4351
8,0.8914,1.5939,1.2625,0.4964,0.4781,0.4222


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
stack_5 = stack_models(best_model_5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9032,1.5303,1.2371,0.4641,0.4831,0.4341
1,0.9258,1.5654,1.2511,0.4230,0.5025,0.4400
2,0.9140,1.5367,1.2397,0.4285,0.4896,0.4627
3,0.9155,1.5549,1.2470,0.4447,0.4904,0.4377
4,0.9208,1.6134,1.2702,0.4629,0.4872,0.4527
5,0.8539,1.3672,1.1693,0.4817,0.4877,0.4113
6,0.9414,1.7311,1.3157,0.4224,0.5055,0.4620
7,0.8996,1.4758,1.2148,0.4571,0.4876,0.4316
8,0.8936,1.6077,1.2679,0.4921,0.4759,0.4290


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

# 2016년 예측

In [20]:
dataset3 = pd.read_csv('./hospital_male_0726_test.csv', encoding='cp949') #이 csv 파일은 2016년 데이터까지 포함한 것
dataset3.drop(columns=['Unnamed: 0'], inplace=True)
dataset3 = dataset3[['area','yyyy','mm','dd','frequency','elderly_pop','total_pop','elderly_ratio','avg_age','pm10','avg_rhm','rhm_min_avg_dif','avg_ta','min_max_Ta','avg_ps','min_max_ps']]
dataset3

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2011,12,1,3.0,95929,773278,12.405500,39.3,15.678261,83.354839,16.669545,1.281944,3.845070,1027.575000,2.993750
1,경기,2011,12,1,4.0,444422,6015357,7.388123,36.0,13.474891,65.448276,14.113793,5.360377,5.413208,1025.875000,3.175000
2,경남,2011,12,1,2.0,152283,1664755,9.147472,37.3,16.193461,74.583333,9.783333,6.394872,3.112821,1023.921429,2.578571
3,경북,2011,12,1,6.0,166547,1355727,12.284700,39.5,12.743785,81.000000,14.400000,4.429091,3.338182,1026.137500,2.684000
4,광주,2011,12,1,0.0,55360,725606,7.629485,35.3,8.566667,89.000000,12.066667,6.240000,3.840000,1022.900000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31581,전남,2016,12,31,NaN,158747,951889,16.677050,42.6,34.282500,79.909091,21.753535,2.539189,10.937838,1030.607692,2.573333
31582,전북,2016,12,31,NaN,139233,927505,15.011563,41.4,44.546347,87.722222,15.777778,1.012903,9.300000,1030.586667,3.466667
31583,제주,2016,12,31,NaN,36378,322545,11.278426,39.2,24.247294,67.741935,22.135484,3.997143,7.931429,1029.900000,3.046667
31584,충남,2016,12,31,NaN,146606,1064765,13.768860,40.7,43.536458,82.550000,17.345000,2.663636,7.596970,1030.100000,2.520000


In [25]:
dataset4 = dataset3[dataset3['yyyy'].astype(str).str.contains('2016')].reset_index(drop=True)#2016년만 추출
dataset4

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2016,1,1,NaN,109543,781185,14.022671,41.4,54.629545,73.529412,19.665950,-1.454167,10.104167,1030.230000,8.010000
1,경기,2016,1,1,NaN,565488,6306634,8.966558,38.0,79.033906,80.588235,20.108824,-0.091837,10.661702,1031.633333,7.016667
2,경남,2016,1,1,NaN,185840,1695187,10.962802,39.3,55.186508,69.500000,31.638889,0.815000,13.610000,1031.175000,5.616667
3,경북,2016,1,1,NaN,195135,1356568,14.384461,41.5,62.858333,68.962963,28.396296,0.371429,13.244643,1030.309524,6.233333
4,광주,2016,1,1,NaN,69876,729995,9.572120,37.4,57.875000,85.333333,23.400000,1.300000,10.300000,1032.200000,5.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,2016,12,31,NaN,158747,951889,16.677050,42.6,34.282500,79.909091,21.753535,2.539189,10.937838,1030.607692,2.573333
6218,전북,2016,12,31,NaN,139233,927505,15.011563,41.4,44.546347,87.722222,15.777778,1.012903,9.300000,1030.586667,3.466667
6219,제주,2016,12,31,NaN,36378,322545,11.278426,39.2,24.247294,67.741935,22.135484,3.997143,7.931429,1029.900000,3.046667
6220,충남,2016,12,31,NaN,146606,1064765,13.768860,40.7,43.536458,82.550000,17.345000,2.663636,7.596970,1030.100000,2.520000


In [26]:
dataset5 = dataset4.drop(columns=['frequency'])

In [27]:
final_model = finalize_model(blender_5)
prediction = predict_model(final_model, data = dataset5)

In [28]:
prediction[['Label']]

,Label
0,1.619783
1,4.580471
2,2.455415
3,2.247964
4,1.051140
...,...
6217,1.414617
6218,1.307677
6219,0.229918
6220,1.218134


In [29]:
save_model(final_model, 'my_best_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=/var/folders/f5/x0_185757lld2fhv6ygmb8f40000gn/T/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['yyyy', 'mm', 'dd', 'elderly_pop',
                                              'total_pop', 'elderly_ratio',
                                              'avg_age', 'pm10', 'avg_rhm',
                                              'rhm_min_avg_dif', 'avg_ta',
                                              'min_max_Ta', 'avg_ps',
                                              'min_max_ps'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  Tra...
                 ('actual_estimator',
                  VotingRegressor(estimators=[('Extra Trees Regressor',
                                               ExtraTreesRegressor(n_jobs=-1,
                                                                   random_state=8699)),
                     

### To load the model back in the environment:

In [30]:
loaded_model = load_model('my_best_pipeline')
print(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=Memory(location=/var/folders/f5/x0_185757lld2fhv6ygmb8f40000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['yyyy', 'mm', 'dd', 'elderly_pop',
                                             'total_pop', 'elderly_ratio',
                                             'avg_age', 'pm10', 'avg_rhm',
                                             'rhm_min_avg_dif', 'avg_ta',
                                             'min_max_Ta', 'avg_ps',
                                             'min_max_ps'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 Tra...
                ('actual_estimator',
                 VotingRegressor(estimators=[('Extra Trees Regressor',
                                              ExtraTreesRegressor(n_jobs=-1,
                                                                  ran

In [31]:
hap = pd.concat([dataset4[['area','yyyy','mm','dd']], prediction['Label']], axis=1)
hap['sex'] = 1
hap

,area,yyyy,mm,dd,Label,sex
0,강원,2016,1,1,1.619783,1
1,경기,2016,1,1,4.580471,1
2,경남,2016,1,1,2.455415,1
3,경북,2016,1,1,2.247964,1
4,광주,2016,1,1,1.051140,1
...,...,...,...,...,...,...
6217,전남,2016,12,31,1.414617,1
6218,전북,2016,12,31,1.307677,1
6219,제주,2016,12,31,0.229918,1
6220,충남,2016,12,31,1.218134,1


In [32]:
verify = pd.read_csv('./edited_data/2-2_검증데이터셋.csv', encoding='cp949')
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [33]:
verify.yyyymmdd[1]

'2016-01-01'

In [34]:
hap

,area,yyyy,mm,dd,Label,sex
0,강원,2016,1,1,1.619783,1
1,경기,2016,1,1,4.580471,1
2,경남,2016,1,1,2.455415,1
3,경북,2016,1,1,2.247964,1
4,광주,2016,1,1,1.051140,1
...,...,...,...,...,...,...
6217,전남,2016,12,31,1.414617,1
6218,전북,2016,12,31,1.307677,1
6219,제주,2016,12,31,0.229918,1
6220,충남,2016,12,31,1.218134,1


In [35]:
hap['yyyy'] = hap['yyyy'].astype('int')
hap['mm'] = hap['mm'].astype('int')
hap['dd'] = hap['dd'].astype('int')

In [36]:
hap['dateInt']=hap['yyyy'].astype(str) + hap['mm'].astype(str).str.zfill(2)+ hap['dd'].astype(str).str.zfill(2)
hap['Date'] = pd.to_datetime(hap['dateInt'], format='%Y%m%d')
hap.drop(columns=['dateInt'], inplace=True)
hap

,area,yyyy,mm,dd,Label,sex,Date
0,강원,2016,1,1,1.619783,1,2016-01-01
1,경기,2016,1,1,4.580471,1,2016-01-01
2,경남,2016,1,1,2.455415,1,2016-01-01
3,경북,2016,1,1,2.247964,1,2016-01-01
4,광주,2016,1,1,1.051140,1,2016-01-01
...,...,...,...,...,...,...,...
6217,전남,2016,12,31,1.414617,1,2016-12-31
6218,전북,2016,12,31,1.307677,1,2016-12-31
6219,제주,2016,12,31,0.229918,1,2016-12-31
6220,충남,2016,12,31,1.218134,1,2016-12-31
